# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Import libraries for Azure Machine Learning SDK

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as pyplot
import numpy as pd
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check the core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Initialize Workspace and Experiment

In [ ]:
# initialized a workspace from a persisted configuration
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'auto-ml-heart-failure-exp'

experiment = Experiment(ws, experiment_name)

# get workspace details and print
output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group 
output['Location'] = ws.location 
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
output_df = pd.DataFrame(data = output, index=[''])
output_df.T

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

## Prepare the dataset for Automation

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "heart-failure-dataset"
description_text = "heart failure clinical records dataset from UCI repo "

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

## Create a Compute Target

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = 'cpu-cluster'

try:
    # check for existing compute target
    training_cluster = ComputeTarget(
        workspace=ws,
        name=cluster_name,
    )
    print('Found existing cluster, use it.')
except:
    # create compute target if non exists
    try:
        compute_config = AmlCompute.provisioning_configuration(
            vm_size='STANDARD_DS11_V2',
            max_nodes=2
        )
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig()

automl_settings = {
    "n_cross_validation": 5,
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 2,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# get the best model
best_model, fitted_model = remote_run.get_output()

# display the properties of the model


In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service